In [1]:
import tensorflow as tf

INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

def forward(x, regularizer):
    w1 = get_weight([INPUT_NODE, LAYER1_NODE], regularizer)
    b1 = get_bias([LAYER1_NODE])
    w2 = get_weight([LAYER1_NODE, OUTPUT_NODE], regularizer)
    b2 = get_bias([OUTPUT_NODE])
    
    a = tf.nn.relu(tf.matmul(x, w1) + b1)
    y = tf.matmul(a, w2) + b2
    return y

def get_weight(shape, regularizer):
    w = tf.Variable(tf.truncated_normal(shape, stddev=0.1), dtype=tf.float32)
    if regularizer != None:
        tf.add_to_collection("losses", tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w

def get_bias(shape):
    b = tf.Variable(tf.zeros(shape))
    return b

In [2]:
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data
import code_12_mnist_forward

STEPS = 50000
BATCH_SIZE = 200
REGULARIZER = 0.0001
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.99
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = "./model/"
MODEL_NAME = "mnist_model"

def backward(mnist):
    x = tf.placeholder(tf.float32, shape=(None, code_12_mnist_forward.INPUT_NODE))
    y = tf.placeholder(tf.float32, shape=(None, code_12_mnist_forward.OUTPUT_NODE))
    
    y_hat = code_12_mnist_forward.forward(x, REGULARIZER)
    global_step = tf.Variable(0, trainable=False)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=tf.argmax(y, 1)))
    loss_total = loss + tf.add_n(tf.get_collection("losses"))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, 
                                               global_step, 
                                               mnist.train.num_examples / BATCH_SIZE, 
                                               LEARNING_RATE_DECAY, 
                                               staircase=True)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_total, global_step=global_step)
    
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    ema_op = ema.apply(tf.trainable_variables())
    with tf.control_dependencies([train_step, ema_op]):
        train_op = tf.no_op(name="train")
        
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        
        for i in range(STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_val, step = sess.run([train_op, loss_total, global_step], feed_dict={x:xs, y:ys})
            if i % 1000 == 0:
                print("After %6d steps, loss is %f" % (step, loss_val))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)
                
def main():
    mnist = input_data.read_data_sets("../data/", one_hot=True)
    backward(mnist)
    
if __name__ == "__main__":
    main()

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import code_12_mnist_forward
import code_12_mnist_backward

TEST_INTERVEL_SEC = 10

def test(mnist):
    with tf.Graph().as_default() as g:  # 使用tf.Graph()复现计算图
        x = tf.placeholder(tf.float32, [None, code_12_mnist_forward.INPUT_NODE])
        y = tf.placeholder(tf.float32, [None, code_12_mnist_forward.OUTPUT_NODE])
        y_hat = code_12_mnist_forward.forward(x, None)
        
        ema = tf.train.ExponentialMovingAverage(code_12_mnist_backward.MOVING_AVERAGE_DECAY)
        ema_restore = ema.variables_to_restore()
        saver = tf.train.Saver(ema_restore)
        
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_hat, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(code_12_mnist_backward.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path) # 恢复模型到当前会话
                    global_step = ckpt.model_checkpoint_path.split("/")[-1].split("-")[-1]
                    accuracy_val = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
                    print("After training %s steps, test accuracy is %f" % (global_step, accuracy_val))
                else:
                    print("No checkpoint file find")
                    return
            time.sleep(TEST_INTERVEL_SEC)
            
def main():
    mnist = input_data.read_data_sets("../data/", one_hot=True)
    test(mnist)
    
if __name__ == "__main__":
    main()